In [6]:
base_model = "egonrp/gpt2-small-portuguese"
chat_file_name = "normais_notime_cleaned_lower.txt"

In [7]:
from datasets import load_dataset

history = load_dataset("./", data_files=chat_file_name)

  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenized_data = tokenizer(history['train']['text'])
tokenized_data = [element for row in tokenized_data["input_ids"] for element in row]
tokenizer.vocab_size

50257

In [9]:
from transformers import TFAutoModelWithLMHead
from tensorflow.keras.optimizers import Adam

model = TFAutoModelWithLMHead.from_pretrained(base_model)
model.compile(optimizer=Adam(3e-5))

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_tf_auto.py:646: FutureWarning: The class `TFAutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `TFAutoModelForCausalLM` for causal language models, `TFAutoModelForMaskedLM` for masked language models and `TFAutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at egonrp/gpt2-small-portuguese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you

In [10]:
import torch
import numpy as np

batch_size = 32
block_size = 64

def get_batch(data):
    while True:
        ix = torch.randint(len(data) - block_size, (batch_size,))
        x = np.array([data[i:i+block_size] for i in ix])
        y = np.array([data[i+1:i+block_size+1] for i in ix])
        yield x, y

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint
# from tensorflow.python.keras.callbacks import ModelCheckpoint

model_file="./cp.ckpt"
cp_callback = ModelCheckpoint(filepath=model_file,
                                        save_weights_only=True,
                                        verbose=1)

In [11]:
model.fit(
    # get_batch(tokenized_data[:19000]),
    # validation_data=get_batch(tokenized_data[19000:]),
    get_batch(tokenized_data[:19000]),
    steps_per_epoch=120,
    epochs=10,
    callbacks=[cp_callback]
)

Epoch 1/10
120/120 [==============================] - ETA: 0s - loss: 5.4979 

KeyboardInterrupt: ignored

In [ ]:
text = "Pedin: Boa tarde grupo"
inputs = tokenizer(text, return_tensors="pt")

# model output using Top-k sampling text generation method
sample_outputs = model.generate(inputs.input_ids,
                                pad_token_id=50256,
                                do_sample=True, 
                                max_length=50, # put the token number you want
                                top_k=40,
                                num_return_sequences=1)

# generated sequence
for i, sample_output in enumerate(sample_outputs):
    print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist())))